In [78]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


In [79]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [80]:
encoder = LabelEncoder()
final_vectors = user_vectors.copy()
x = encoder.fit(final_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
username_enc

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc

#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])
final_vectors

,username,experience,gender,generation,fav_anime_period,Slice,Dementia,Arts,Adventure,Vampire,Comedy,Shounen,Sports,Power,Military,Space,Mecha,Music,Supernatural,Mystery,Horror,Kids,Cars,Harem,School,Shoujo,Parody,Psychological,Action,Sci-Fi,Game,Fantasy,Martial,Romance,Ai,Life,Drama,of,Ecchi,Police,Historical,Super,Magic,Thriller,Seinen
0,7537,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1179,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,6556,3,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,5037,3,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,3575,3,0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9297,8352,3,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9298,2558,3,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9299,4967,3,1,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9300,9113,2,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Compute K-Nearest Neighbours

In [81]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours(user):
    #user = np.array([7537, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data
    n = train.kneighbors([user], return_distance = False)
    return n

#returns a dataframe of users that were in k-nearest neighbours list 
def sim_frame(neighbours):
    sim_names = []
    #neighbours = neighbours()

    for i in neighbours[0]: #loop through nested array
        #print(final_arrays[i][0])
        name = x.inverse_transform([final_arrays[i][0]]) #find usernames of similar users
        sim_names.append(name)

    #build new dataframe of neighbours
    neighbour_frame = pd.DataFrame()
    for i in users.columns:
        neighbour_frame[i] = None

    for name in sim_names:
        user_row = users.loc[users["username"]==name[0]]
        neighbour_frame = pd.concat([neighbour_frame, user_row])

    pd.options.display.max_rows = 999
    pd.set_option('display.max_columns', 500)
    return neighbour_frame

neighbour_ids = neighbours(np.array([7537, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
neighbour_frame = sim_frame(neighbour_ids)
neighbour_frame
#final_vectors.loc[final_vectors["username"]==7537]
#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
0,2255153,karthiga,49,Active,Female,1990,3391,Millenials,"['Comedy', 'Shoujo', 'Romance']",Classic
7633,108217,marix_yume,118,Active,Female,1991,5138,Millenials,"['Shoujo', 'Romance', 'Comedy']",Classic
4654,82687,nexi,149,Active,Female,1989,4784,Millenials,"['Comedy', 'Romance', 'Shoujo']",Classic
8295,79420,crusnikgirl,48,Active,Female,1993,3317,Millenials,"['Comedy', 'Shoujo', 'Romance']",Classic
7132,487787,zebrasXP,275,Active,Female,1993,5091,Millenials,"['Romance', 'Shoujo', 'Comedy']",Classic


Create anime recommendations

In [82]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours, media_type):
    #Get top rated TV shows or movies from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        tv_shows = user_ratings[user_ratings["type"]=="TV"]
        movies = user_ratings[user_ratings["type"]=="Movie"]

        if media_type == "TV":
            show_ratings = tv_shows
        else:
            show_ratings = movies
        
        #There may be 0 movies in the user's top 10 list in which case we skip that user's recommendationan
        if len(show_ratings) > 1:
            top_rated = show_ratings.iloc[0]["title"]
            suggestions.append(top_rated)
            #print(top_rated)

    if len(suggestions)==0:
        print("No movies to recommend :/")
    
    return suggestions

recommendations(neighbour_frame, "TV")
#ratings

['Kamisama Hajimemashita',
 'D.Gray-man',
 'Gakuen Utopia Manabi Straight!',
 'One Piece',
 'Bishoujo Senshi Sailor Moon SuperS']

Evaluation of the recommended anime on all users in the test set

In [83]:
#Create kneighbours dataframe for all users in the dataset
for user in final_vectors.values:
    user = np.array(user)
    user_neighbours = neighbours(user)
    nframe = sim_frame(user_neighbours)
    #print(user)
    #print(user_neighbours)
    user_recommendations = recommendations(nframe, "TV")
    print(nframe["username"], user_recommendations)
    #1. now just find the shows that all neighbours watched
    #2. check if recommended shows are among them
    #3. check what the user rated the show
    #4. present results

#include analysis of distance metric between neighbours
#(final_vectors.columns)    

0          karthiga
7633     marix_yume
4654           nexi
8295    crusnikgirl
7132       zebrasXP
Name: username, dtype: object ['Kamisama Hajimemashita', 'D.Gray-man', 'Gakuen Utopia Manabi Straight!', 'One Piece', 'Bishoujo Senshi Sailor Moon SuperS']
1         Damonashu
2785       Danumaki
8266     DarkMoonbg
4248    Cornholio77
6390       Connacht
Name: username, dtype: object ['One Piece', 'Full Metal Panic!', 'Macross F', 'Angel Beats!', 'Neon Genesis Evangelion']
2                 bskai
2846            cotofei
1480             Thowil
2953           rdgneoz3
5740    animefanforlife
Name: username, dtype: object ['Code Geass: Hangyaku no Lelouch', 'Nodame Cantabile', 'Higashi no Eden', 'Sayonara Zetsubou Sensei', 'Shugo Chara!! Doki']
3         Slimak
6986    Striefer
7394     PrinzkO
7874     Tauryna
3141     Myokuro
Name: username, dtype: object ['Shingeki no Kyojin Season 2', 'Last Exile', 'Code Geass: Hangyaku no Lelouch', 'Guilty Crown', 'Clannad: After Story']
4       Mist

KeyboardInterrupt: 

Function that converts user information from website into vector